In [1]:
# 결론: word2vec이라는건 그냥 각 검색어에 대해서 labeling을 하기 위함이다.
# 고민해야할 것: 검색어에 대해서 label이 주어졌을 때, 이걸 어떻게 활용할 수 있을까?

# Problem 1)
# Word2Vec: 띄어쓰기가 없음
# 검색어: 띄어쓰기가 있음
# => 어떻게 해야할까....?????

# 배트맨리부트 o.k
# 베수비오화산 o.k
# 배트맨비긴즈명대사 희박함
# 배트맨 | 비긴즈 | 명대사

# 챗봇이랑 비슷함
# 정확한 목적으로 연결시켜줌

# Problem 2)
# 데이터 라벨이 되어도, 결측치가 있을 수 있음.

# Answer 2)
# 결측치 자체를 새로운 라벨로 정의함.
# 엄청 많은 데이터를 수집해야함
# 실제 매칭되지 않은 (결측치) 것들의 개수(비율)을 파악 1325252개 중 5% 결측치 정도면 새로운 클러스터 ㄱㅊ... 근데 25%이러면 쓰기 힘듦. 개선이 필요하거나, 반려해야함.
# Ex) 1 ~ 30 클러스터를 만들었음. 새로운 단어는 기존 단어장에 없음. 얘를 `31번째 클러스터`를 만들어서 안에 넣음.

In [1]:
import pandas as pd
import urllib

In [2]:
# train_profiles = pd.read_excel('train_profiles.xlsx')
train_searchkeywords_origin = pd.read_table('train_searchkeywords.tab', encoding = 'cp949')
# train_clickstream = pd.read_table('train_clickstreams.tab', encoding = 'cp949')

In [17]:
train_searchkeywords_origin

,CUS_ID,QRY_STR,QRY_CNT
0,1,못내,1.0
1,1,배트맨 리부&acr=1&qdt=0&ie=utf8&query=배트맨 리부트,1.0
2,1,배트맨 비긴즈 명대사&sm=top_sug.pre&fbm=0&acr=9&acq=배트맨...,1.0
3,1,베수비&acr=1&qdt=0&ie=utf8&query=베수비오 화산,1.0
4,1,베이징올림픽 장미란,1.0
...,...,...,...
1325247,999,고래&sm=top_sug.pre&fbm=1&acr=1&acq=고래,1.0
1325248,999,고래,1.0
1325249,999,현무,2.0
1325250,999,허영심,1.0


In [3]:
def query_string_refinement(query_str):
    """
    Return
    ======
    &query in string => return something
    else => return the first word
    """
    if '&query' not in query_str:
        return query_str.split('&')[0]
    return urllib.parse.parse_qs(query_str)['query'][0]

In [4]:
train_searchkeywords = train_searchkeywords_origin.copy()

In [5]:
train_searchkeywords['KEYWORD'] = train_searchkeywords['QRY_STR'].apply(lambda x: query_string_refinement(x))
train_searchkeywords['WORD_LEN'] = train_searchkeywords['KEYWORD'].apply(len)
train_searchkeywords['WORD_SPLIT'] = train_searchkeywords['KEYWORD'].apply(lambda x: x.split(' '))
train_searchkeywords['DIFF_WORD'] = train_searchkeywords['WORD_SPLIT'].apply(lambda x: len(x))
train_searchkeywords['SEARCH_ID'] = train_searchkeywords.index
train_searchkeywords = train_searchkeywords.drop(columns = ['QRY_STR'])
# train_searchkeywords = train_searchkeywords.explode('WORD_SPLIT')

In [6]:
train_searchkeywords

,CUS_ID,QRY_CNT,KEYWORD,WORD_LEN,WORD_SPLIT,DIFF_WORD,SEARCH_ID
0,1,1.0,못내,2,[못내],1,0
1,1,1.0,배트맨 리부트,7,"[배트맨, 리부트]",2,1
2,1,1.0,배트맨 비긴즈 명대사,11,"[배트맨, 비긴즈, 명대사]",3,2
3,1,1.0,베수비오 화산,7,"[베수비오, 화산]",2,3
4,1,1.0,베이징올림픽 장미란,10,"[베이징올림픽, 장미란]",2,4
...,...,...,...,...,...,...,...
1325247,999,1.0,고래,2,[고래],1,1325247
1325248,999,1.0,고래,2,[고래],1,1325248
1325249,999,2.0,현무,2,[현무],1,1325249
1325250,999,1.0,허영심,3,[허영심],1,1325250


In [7]:
import pickle as pkl

with open('model.pkl', 'rb') as f:
    model = pkl.load(f)
    vocab = pkl.load(f)

In [8]:
import numpy as np

def get_vector(word_list):
    whole_word = np.zeros(1000)
    for word in word_list:
        try:
            word_vec = model.wv.get_vector(word)
        except:
            continue
        whole_word += word_vec
    return whole_word

In [9]:
train_searchkeywords['KEYWORD_VECTOR'] = train_searchkeywords['WORD_SPLIT'].apply(lambda x: get_vector(x))

In [10]:
from sklearn.cluster import KMeans

In [11]:
kmeans = KMeans(n_clusters = 40)

In [12]:
kmeans.fit(model.wv[model.wv.vocab])

KMeans(n_clusters=40)

In [13]:
test = np.concatenate(train_searchkeywords['KEYWORD_VECTOR'])

In [14]:
test = test.reshape(1325252, 1000)

In [16]:
test.astype('float32')

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03298732,  0.01132629, -0.06086851, ...,  0.05046278,
        -0.00790973, -0.01310975],
       [ 0.03298732,  0.01132629, -0.06086851, ...,  0.05046278,
        -0.00790973, -0.01310975],
       ...,
       [ 0.02488056, -0.09300306, -0.05450519, ...,  0.0057012 ,
         0.03933817, -0.05928488],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [15]:
test2 = np.array(test, dtype = np.float32)

KeyboardInterrupt: 

In [ ]:
train_searchkeywords['label'] = kmeans.predict(test2)

In [ ]:
train_searchkeywords['label'].value_counts()

In [ ]:
zero_vec_cond = train_searchkeywords['KEYWORD_VECTOR'].apply(lambda x: np.array_equal(x, np.zeros(1000)))

In [22]:
train_searchkeywords.loc[zero_vec_cond, 'label'] = -1

In [23]:
keyword_gp = train_searchkeywords.groupby(['CUS_ID', "label"])
object_column = [
   'TOTAL_SEARCH_COUNT',
   'UNIQUE_SEARCH_COUNT',
   'WORD_LENGTH_AVG_AT_ONCE',
   'WORD_COUNT_AVG_AT_ONCE',
]
train_keyword_df = pd.DataFrame(data = [
                                    keyword_gp['QRY_CNT'].sum(),
                                    keyword_gp['KEYWORD'].nunique(),
                                    keyword_gp['WORD_LEN'].mean(),
                                    keyword_gp['DIFF_WORD'].mean()
                                ],
                                index = object_column)\
                     .transpose()\
                     .reset_index()

In [24]:
for col in object_column:
    col_df = pd.pivot_table(
                    train_keyword_df, 
                    index = 'CUS_ID', 
                    columns = ['label'], 
                    values = [
                        col
                    ]
                )[col]
    col_df_col = col_df.columns
    col_df.columns = [f"{col}_FOR_LABEL_{label}" for label in col_df_col]
    print(col_df)
    try:
        total_df = total_df.join(col_df)
    except:
        total_df = col_df

        TOTAL_SEARCH_COUNT_FOR_LABEL_-1  TOTAL_SEARCH_COUNT_FOR_LABEL_0  \
CUS_ID                                                                    
1                                 176.0                             7.0   
2                                1204.0                            15.0   
3                                 135.0                             1.0   
4                                 113.0                             1.0   
5                                 375.0                             6.0   
...                                 ...                             ...   
2496                              171.0                             2.0   
2497                              290.0                             1.0   
2498                              659.0                             1.0   
2499                              449.0                             6.0   
2500                              132.0                             8.0   

        TOTAL_SEARCH_COU

In [25]:
total_df = total_df.fillna(0).reset_index()

In [26]:
total_df

,CUS_ID,TOTAL_SEARCH_COUNT_FOR_LABEL_-1,TOTAL_SEARCH_COUNT_FOR_LABEL_0,TOTAL_SEARCH_COUNT_FOR_LABEL_1,TOTAL_SEARCH_COUNT_FOR_LABEL_2,TOTAL_SEARCH_COUNT_FOR_LABEL_3,TOTAL_SEARCH_COUNT_FOR_LABEL_4,TOTAL_SEARCH_COUNT_FOR_LABEL_5,TOTAL_SEARCH_COUNT_FOR_LABEL_6,TOTAL_SEARCH_COUNT_FOR_LABEL_7,...,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_30,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_31,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_32,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_33,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_34,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_35,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_36,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_37,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_38,WORD_COUNT_AVG_AT_ONCE_FOR_LABEL_39
0,1,176.0,7.0,26.0,4.0,5.0,29.0,5.0,6.0,2.0,...,0.000000,3.333333,5.000000,2.166667,2.000000,3.000000,2.0,2.250000,2.714286,2.333333
1,2,1204.0,15.0,11.0,4.0,3.0,183.0,4.0,4.0,23.0,...,3.000000,2.153846,2.000000,1.666667,1.454545,1.000000,3.0,2.142857,1.666667,2.000000
2,3,135.0,1.0,4.0,2.0,3.0,27.0,1.0,3.0,3.0,...,0.000000,2.250000,0.000000,3.000000,2.750000,0.000000,2.0,0.000000,2.250000,2.333333
3,4,113.0,1.0,1.0,0.0,0.0,7.0,0.0,0.0,1.0,...,2.000000,2.000000,0.000000,2.333333,3.000000,0.000000,0.0,0.000000,1.500000,1.000000
4,5,375.0,6.0,3.0,6.0,4.0,17.0,10.0,2.0,13.0,...,2.857143,2.769231,3.400000,3.250000,1.833333,0.000000,0.0,3.000000,1.666667,3.391304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,2496,171.0,2.0,0.0,0.0,0.0,7.0,0.0,5.0,6.0,...,0.000000,2.142857,2.000000,3.000000,2.000000,3.000000,2.0,0.000000,2.000000,0.000000
2457,2497,290.0,1.0,18.0,2.0,2.0,22.0,0.0,3.0,3.0,...,2.000000,2.600000,2.333333,3.000000,2.250000,2.000000,4.0,2.666667,2.500000,3.500000
2458,2498,659.0,1.0,3.0,0.0,19.0,51.0,0.0,4.0,2.0,...,2.500000,2.588235,0.000000,2.571429,1.000000,2.285714,2.0,2.000000,2.500000,2.000000
2459,2499,449.0,6.0,11.0,3.0,3.0,55.0,2.0,8.0,8.0,...,3.000000,2.500000,2.000000,2.000000,1.666667,0.000000,2.0,2.500000,1.909091,2.555556


In [27]:
test.shape

(1325252, 1000)

In [28]:
test[1][0]

0.0329873226583004

In [29]:
import pickle as pkl
with open("total_dataframe.pkl", 'rb') as f:
    data = pkl.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'total_dataframe.pkl'

In [ ]:
data.columns